In [1]:
import pandas as pd
import glob
import warnings
warnings.filterwarnings('ignore')
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import json
from LabelAlis import LabelAlis
from scipy.stats import spearmanr

Coverage_Of_PfamChoppedStruct = 0
Coverage_Of_PfamOnQueryPosThresh = 0.25
FalsePositiveThreshold = 0.25

In [4]:
def SplitTargetSeedInfo(PdSeries):
    AlnPfDf = PdSeries.str.split("_", expand=True)
    AlnPfDf.columns = ["targetprot", "targetprotstart","targetprotend","predpf"]
    AlnPfDf["targetprotstart"] = AlnPfDf["targetprotstart"].astype(int)
    AlnPfDf["targetprotend"] = AlnPfDf["targetprotend"].astype(int)
    return AlnPfDf

In [5]:
def ExpandSeedAnns(AlnPfDf, pfClans):
    AlnPfDf = pd.concat([AlnPfDf, SplitTargetSeedInfo(AlnPfDf['target']) ], axis=1) 
    return AlnPfDf

In [6]:
def filter_bits(df, bits):
    return df[df["bits"]>= bits]
def filter_bits_alnlen(df, bits_alnlen):
    return df[df["bits__alnlen"]>=bits_alnlen]

In [7]:
def select_filter(filapth):
    if filepath.endswith("cut.tsv"):
        func = lambda x: filter_bits(x, 152)
    elif filepath.endswith("alnlen.tsv"):
        func = lambda x: filter_bits_alnlen(x, 0.86)
    else:
        func = None
    return func

In [8]:
def Labeler(inputfile, clanassociation, pf):
    filt = select_filter(inputfile)
    df_raw = pd.read_csv(inputfile, sep="\t")
    if filt!=None:
        df = filt(df_raw)
    else:
        df = df_raw
    df = ExpandSeedAnns(df, clanassociation)
    processed_df = df.merge(clanassociation, left_on = "predpf", right_on = "pf",how="left").drop("pf", axis=1).rename(columns={"clan":"predclan"})
    Labeled_my= LabelAlis(processed_df, pf, clanassociation, ["qstart", "qend"], how2merge="left")
    prefix = pf.columns[1]
    Labeled_my = Labeled_my[(Labeled_my["isseed"]!=1 )] #|(Labeled_my["equalto"+ prefix + "preds"]==-1 )] ####NOTE: I am discarding Seed annotations for this plot
    label = pf.columns[1]
    label_dict = {0: "FP", 1: "TP", -1: "FN"}
    Labeled_my['labelcol_str']= Labeled_my["equalto{}preds".format(label)].map(label_dict)
    #Labeled_my = Labeled_my[Labeled_my["bits"]>=152]
    return Labeled_my

In [9]:
clanassociation = pd.read_csv("../PfamClanAssociation.tsv",header=None, names = ["pf", "clan"], sep="\t" )
pf = pd.read_csv("../PfamTb_SeedMeanlddtSize.txt",sep="\t")
pfn = pd.read_csv("../PfamNTb_SeedMeanlddtSize.txt",sep="\t")

In [3]:
files = ["../greedy_hits/aln_Tb_pf_fscut.tsv", "../greedy_hits/aln_Tb_pf_pdbcut.tsv",
        "../greedy_hits/aln_Tb_pf_fscut_bits__alnlen.tsv", "../greedy_hits/aln_Tb_pf_pdbcut_bits__alnlen.tsv",
        "../greedy_hits/aln_Tb_pf_seq_gr.tsv"]

In the following cell, we will find the hypothetical proteins in T. brucei

In [51]:
GeneDesc = pd.read_csv("../TriTrybID_description.tsv", header=None, sep="\t")
GeneDesc[1] = GeneDesc[1].str.lower()
mapping = pd.read_csv("../MappingFromUnipToTriTrypID.txt", sep="\t", header=None)
DescUnip = GeneDesc.merge(mapping, left_on = 0, right_on = 0).drop(columns=[0])[["1_y", "1_x"]].rename(columns={"1_y":"unipid", "1_x":"description"})
HypotheticalProteins = GeneDesc[GeneDesc[1].str.contains("hypothetical protein")].merge(mapping, on = [0])
HypProts_Unip = HypotheticalProteins[["1_y", "1_x"]].rename(columns= {"1_x": "description", "1_y": "unipid"})

In [57]:
labeled_data = {}
for filepath in files:
    df_pf = Labeler(filepath, clanassociation, pf)
    df_name = os.path.basename(filepath).replace(".tsv", "")
    labeled_data[df_name] = df_pf
    new_agpf = df_pf[df_pf["equaltopfampreds"]==-1]
    df_pfn= Labeler(filepath, clanassociation, pfn)
    new_agpfn= df_pfn[df_pfn["equaltopfampreds"]==-1]
    new_both = new_agpf[["query", "target"]].merge(new_agpfn)
    
    new_both_mostfreq = new_both["predpf"].value_counts()[0]
    new_both_mostfreqpf = new_both["predpf"].value_counts().index[0]
    
    mean_len = (new_both["qend"] - new_both["qstart"]).mean() + 1
    
    fident_mean = new_both["fident"].mean()
    
    HypNum = HypProts_Unip.merge(new_both, left_on = "unipid", right_on = "query" )
    
    print(f"{df_name}\t{new_agpf.shape[0]}\t{new_agpfn.shape[0]}\t{new_both.shape[0]}\t{round(mean_len,1)}\t{new_both_mostfreqpf}\t{new_both_mostfreq}\t{round(fident_mean,2)}\t{HypNum.shape[0]}")   

aln_Tb_pf_fscut	1251	600	470	232.6	PF13458	35	0.17	230
aln_Tb_pf_pdbcut	1108	514	408	254.3	PF13458	31	0.15	197
aln_Tb_pf_fscut_bits__alnlen	998	526	390	127.3	PF12894	13	0.21	185
aln_Tb_pf_pdbcut_bits__alnlen	774	401	290	136.3	PF10358	9	0.18	148
aln_Tb_pf_seq_gr	336	341	156	68.1	PF00560	69	0.73	20
